In [1]:
## Uncomment this section to install required packages

#!pip install astroquery
#!pip install Astropy

In [2]:
from astroquery.gaia import Gaia
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table, join
import scipy.stats as st
meta_source = Gaia.load_table('gaiadr3.gaia_source')
meta_source
print(meta_source)

Important:  Gaia archive will be intermittently unavailable due to scheduled maintenance on 14-10-2024 from 10:30 to 12:30 (CEST)
TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152


In [3]:
# This query selects stars from Gaia DR3 that meet the data quality standards for number and consistancy
# of measurements (Babusiaux C, Leeuwen F van, Barstow MA, Jordi C, Vallenari A, et al. 2018. A&A. 616:A10). 

# The color cuts, "bp_rp >l 1.8 AND bp_rp < 5.0 AND g_rp > 0.92 AND g_rp <1.65" limit the selection to M-dwarf stars
# ("A Modern Mean Dwarf Stellar Coor and Effective Temperature Sequence", 
# http://www.pas.rochester.edu/~emamajek/EEM_dwarf_UBVIJHK_colors_Teff.txt, # Eric Mamajek, # Version 2022.04.16).

# The final temperature and absolute magnitude cuts, "teff_gspphot < 3900 AND phot_g_mean_mag+5*log10(parallax)-10 > 4"
# remove any M-giant stars.

# Stars are drawn from a strip of sky 20-deg wide centered on 180-deg galactic latitude.

query_M = """
    SELECT TOP 250000
    source_id, phot_g_mean_mag+5*log10(parallax)-10 AS mag_abs, 4.74057*pm*distance_gspphot/1000 AS v_tan,
    ra, dec, l, b, distance_gspphot, parallax,
    phot_g_mean_mag, phot_rp_mean_mag, phot_bp_mean_mag, g_rp, bp_rp, ag_gspphot,
    ebpminrp_gspphot, pm, teff_gspphot, logg_gspphot, mh_gspphot,
    pmdec, pmra
    FROM gaiadr3.gaia_source
    WHERE parallax_over_error > 10
    AND phot_g_mean_flux_over_error>50
    AND phot_rp_mean_flux_over_error>20
    AND phot_bp_mean_flux_over_error>20
    AND phot_bp_rp_excess_factor < 1.3+0.060*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2)
    AND visibility_periods_used>8
    AND astrometric_chi2_al/(astrometric_n_good_obs_al-5)<1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))
    AND bp_rp > 1.8 AND bp_rp < 5.0
    AND g_rp > 0.92 AND g_rp <1.65
    AND teff_gspphot < 3900
    AND l>170. AND l<190.
    """

In [ ]:
# Run the query. Using the asynchronus query aviods the HTTP timeout 408 error. 

job_M = Gaia.launch_job_async(query_M, output_format = 'votable')
job_M

In [ ]:
# Convert results to dataframe and display the results.

results_m = job_M.get_results()
m_dwarf_df = results_m.to_pandas()
N_total = len(m_dwarf_df)
m_dwarf_df

In [ ]:
 # Plot and H-R diagram of the M-dwarf stars to verify that the selected stars are on the red dwarf section of the main sequence.

plt.scatter(m_dwarf_df['g_rp'], m_dwarf_df['mag_abs'], s = 0.01, alpha = 1, label = "K-Dwarf", c = m_dwarf_df["teff_gspphot"])
plt.colorbar(label='Temperature [K]')
plt.title('M Dwarf Stars')
plt.xlabel('Color G - rp [mag]')
plt.ylabel('G mag abs [mag]')
plt.xlim(0.8, 1.4)
plt.ylim(14, 6)
plt.grid()
# plt.legend()

In [ ]:
# Tangential velocity histogram

plt.hist(m_dwarf_df['v_tan'], bins=100, density=False)
plt.xlabel('Tangential Velocity [km/s]')
plt.ylabel('Number')
plt.xlim(0,400)
#plt.ylim(0,3000)
plt.title(f'Tangential Velocity Distribution, N = {N_total:.0f}')
plt.grid()
plt.minorticks_on()

# plt.savefig("data/v_tan_hist.png")

In [ ]:
# Find low tangental velocity stars. These are potential disk stars.

low_v_tan = m_dwarf_df[m_dwarf_df['v_tan'] < 50]
N_low_v_tan = len(low_v_tan)
low_v_tan

In [ ]:
# Find high tangental velocity stars. These are potential halo stars.

high_v_tan = m_dwarf_df[m_dwarf_df['v_tan'] > 100]
N_high_v_tan = len(high_v_tan)
high_v_tan

In [ ]:
# Save the files high and low tangential velocity files as csv.

high_v_tan.to_csv('dataset/high_v_tan.csv')
low_v_tan.to_csv('dataset/low_v_tan.csv')

In [ ]:
# Plot an HR diagram of the two groups.

plt.scatter(low_v_tan['g_rp'], low_v_tan['mag_abs'], s = 0.01, alpha = 0.25, label = "Disk")
plt.scatter(high_v_tan['g_rp'], high_v_tan['mag_abs'], s = 0.05, alpha = 0.5, label = "Halo")
# plt.colorbar(label='Metallicity [dex]')
plt.title('M Dwarf Stars, Halo and Disk')
plt.xlabel('Color G - rp [mag]')
plt.ylabel('G mag abs [mag]')
plt.xlim(0.8, 1.4)
plt.ylim(14, 6)
plt.grid()
plt.legend()

#plt.savefig("data/hr_diagram.png")

In [ ]:
# Compare galactic latitude of halo and disk stars

bins = 20
fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
N2=42
axs[0].hist(low_v_tan['b'], bins=bins, density=True)
axs[0].set_xlabel('Latitude [Deg]')
axs[0].set_xlim(-90,90)
#axs[0].set_ylim(0,1.2)
axs[0].set_title(f'M-dwarfs Disk, N = {N_low_v_tan:.0f}')
axs[0].grid()
axs[0].minorticks_on()

axs[1].hist(high_v_tan['b'], bins=bins, density=True)
axs[1].set_xlabel('Latitude [Deg]')
axs[1].set_xlim(-90,90)
#axs[1].set_ylim(0,1.2)
axs[1].set_title(f'M-dwarfs Halo, N = {N_high_v_tan:.0f}')
axs[1].grid()
axs[1].minorticks_on()

plt.savefig("data/v_tan_hist.png")

In [ ]:
# Trim data files to save space. Remove photometry data and keep position data.

high_v_tan_position = high_v_tan.drop(['ra', 'dec', 'distance_gspphot', 'parallax',
    'phot_g_mean_mag', 'phot_rp_mean_mag', 'phot_bp_mean_mag', 'bp_rp', 'ag_gspphot',
    'ebpminrp_gspphot', 'pm', 'teff_gspphot', 'logg_gspphot', 'mh_gspphot',
    'pmdec', 'pmra'], axis=1)
low_v_tan_position = low_v_tan.drop(['ra', 'dec', 'distance_gspphot', 'parallax',
    'phot_g_mean_mag', 'phot_rp_mean_mag', 'phot_bp_mean_mag', 'bp_rp', 'ag_gspphot',
    'ebpminrp_gspphot', 'pm', 'teff_gspphot', 'logg_gspphot', 'mh_gspphot',
    'pmdec', 'pmra'], axis=1)

# Save the files.
high_v_tan_position.to_csv('data/high_v_tan_position.csv')
low_v_tan_position.to_csv('data/low_v_tan_position.csv')